In [2]:
import re

In [3]:
seps = ['-','/',' ','.']
sepRe = re.compile('[-/ \.]')

In [4]:
monthMapping = {'JAN': '01', 'JANUARY': '01',
                'FEB': '02', 'FEBRUARY': '02',
                'MAR': '03', 'MARCH': '03',
                'APR': '04', 'APRIL': '04',
                'MAY': '05',
                'JUN': '06', 'JUNE': '06',
                'JUL': '07', 'JULY': '07',
                'AUG': '08', 'AUG': '08',
                'SEP': '09', 'SEPTEMPBER': '09',
                'OCT': '10', 'OCTOBER': '10',
                'NOV': '11', 'NOVEMBER': '11',
                'DEC': '12', 'DECEMBER': '12'
                   }

In [5]:
def checkYear(date,separator):
    if len(date[:date.find(separator)]) == 4 or len(date[date.find(separator) + 1:]) == 4:
        return True
    else:
        return False

In [6]:
def findYearPos(date,separator):
    return 'begin' if len(date[:date.find(separator)]) == 4 else 'end'

In [7]:
def mapMonth(date,separator):
    
    monthStartPos,monthEndPos = re.search('[a-zA-Z]+',date).start(), re.search('[a-zA-Z]+',date).end()
    month = monthMapping[date[monthStartPos:monthEndPos].upper()]
    
    sepIterator = sepRe.finditer(date)
    sepList = [match.span() for match in sepIterator]
    
    if monthStartPos == 0 and len(sepList) == 2:
        print(date[monthEndPos + 1:date.find(separator)])
        date = date[monthEndPos + 1:sepList[1][0]] + separator + month + date[sepList[1][0]:]
        
    elif monthStartPos == 0 and len(sepList) == 1 and len(date[monthEndPos + 1:]) != 4:
        date = date[monthEndPos + 1] + separator + month
        
    else:
        date = date[:monthStartPos] + month + date[monthEndPos:]
        
    return date

In [8]:
def cleanDate(date):
    index = 0
    for item in ['st', 'nd', 'rd', 'th']:
        if item in date:
            date = date.replace(item,'')
    return date

In [9]:
def guessDateFormat(date,outputFormat):
    if '-' in date:
        separator = '-'
    elif '/' in date:
        separator = '/'
    elif ' ' in date:
        separator = ' '
    else:
        separator = None
        
    date = cleanDate(date)
    
    if re.search('[a-zA-Z]',date) is not None:
        date = mapMonth(date,separator)
    
    if separator == None and len(date)==4:
        actualInputFormat = '%Y'
        actualOutputFormat = '%Y'
    
    elif separator == None and len(date) == 8:
        actualInputFormat = '%d%m%Y'
        actualOutputFormat = outputFormat
    
    elif date.count(separator) == 1:
        yearPresent = checkYear(date,separator)
        
        if yearPresent and findYearPos(date,separator) == 'begin':
            actualInputFormat = '%Y' + separator + '%m'
            actualOutputFormat = '%m-%Y'
            
        elif yearPresent and findYearPos(date,separator) == 'end':
            actualInputFormat = '%m'+ separator + '%Y'
            actualOutputFormat = '%m-%Y'
            
        else:
            actualInputFormat = '%m' + separator + '%d' if int(date[date.find(separator)+1:]) > 12 else '%d' + separator + '%m'
            actualOutputFormat = '%d-%m'
            
    elif len(date[:date.find(separator)]) < 4:
        actualInputFormat = '%d' + separator + '%m' + separator + '%Y'
        actualOutputFormat = '%d-%m-%Y'
        
    else:
        actualInputFormat = '%Y' + separator + '%m' + separator + '%d'
        actualOutputFormat = '%d-%m-%Y'
            
    return actualOutputFormat

In [10]:
def convert_date(date, output_format="%d-%b-%Y"):
    from dateutil.parser import parse
    from datetime import datetime
    try:
        actual_output_format = guessDateFormat(date,output_format)
        pd = parse(date)
        return pd.strftime(actual_output_format)
    except Exception as e:
#         print ("eee")
        for i,fort in enumerate(["%d%m%Y","%Y%m%d","%Y%d%m","%m%d%Y"]):
            try:
#                 print i,fort,date
                d = datetime.strptime(date,fort ).date()
                d.replace( year=d.year-100 )#^2053   ^1953
                d = d.strftime(actual_output_format)
                return d,fort
            
            except Exception as e:
                pass
#                 print "format>>", e
        pass
#         print "E", e
    return 

In [12]:
print(convert_date("06/1987",output_format="%d-%b-%Y"))

06-1987
